In [1]:
%matplotlib inline

import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.plotly as py
import time
import math
import xgboost

from sklearn import neural_network
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_squared_error
from math import sqrt

timeformat = "%Y-%m-%d %H:%M:%S"
locations = ["EL", "LB", "NT", "CH"]

data = pd.read_csv("../data/ensemble.csv")
date = pd.read_csv("../data/date.csv")

data = pd.concat([data,date], axis=1)

# Preprocessing

In [2]:
def to_angle(angle):
    return (angle*math.pi)/180.0

def wind(cell, angle):
    cell = (cell - angle)
    weight = math.cos(to_angle(cell))
    # ReLU
    return weight * (weight > 0)

In [3]:
for i in range(6):
    data['EL_PM25_' + str(i+1)] = data['EL_PM25'].shift(i+1)

In [4]:
# 風向的權重
data['LB_WD_W'] = data.LB_WD.apply(wind, args=(198.39,))
data['CH_WD_W'] = data.CH_WD.apply(wind, args=(40.51,))
data['NT_WD_W'] = data.NT_WD.apply(wind, args=(92.71,))

In [5]:
# 與二林PM25的差值
data['LB_D_PM25'] = data.LB_PM25 - data.EL_PM25
data['NT_D_PM25'] = data.NT_PM25 - data.EL_PM25
data['CH_D_PM25'] = data.CH_PM25 - data.EL_PM25

In [6]:
# 目標值
data['target'] = data.EL_PM25.shift(-1)

# Normalization

# Select feature

In [7]:
PM25 = ["EL_PM25"]
hour_cols = [f for f in data.columns.tolist() if "h_" in f]
month_cols = [f for f in data.columns.tolist() if "m_" in f]
exclude_cols = ['datetime', 'target']
meteorology_cols = ['EL_TEMP', 'EL_RH', 'EL_RAINFALL', 'EL_WS']
lags_cols = [f for f in data.columns.tolist() if 'EL_PM25_' in f]
wd_cols = ["LB_WD_W", "CH_WD_W", "NT_WD_W"]
neighbors_cols = ["LB_PM25", "CH_PM25", "NT_PM25"]
neighbors_D_cols = ["LB_D_PM25", "CH_D_PM25", "NT_D_PM25"]

# Season Clustering

# Train & Test Split

In [8]:
start = data[data.datetime.values == '2016-01-01 00:00:00'].index[0]

datatrain = data[:start]
datatest = data[start:]

column = PM25 + exclude_cols + meteorology_cols + lags_cols + hour_cols + wd_cols + neighbors_D_cols

train = datatrain[column].dropna()
test = datatest[column].dropna()

A_train, A_test = train[PM25 + hour_cols + lags_cols], test[PM25 + hour_cols + lags_cols]
y_train, y_test = train["target"], test["target"]
B_train, B_test = train[PM25 + neighbors_D_cols], test[PM25 + neighbors_D_cols]

In [9]:
print("A Model:\n" + ", ".join(A_train.columns.tolist()))

A Model:
EL_PM25, h_0, h_1, h_2, h_3, h_4, h_5, h_6, h_7, h_8, h_9, h_10, h_11, h_12, h_13, h_14, h_15, h_16, h_17, h_18, h_19, h_20, h_21, h_22, h_23, EL_PM25_1, EL_PM25_2, EL_PM25_3, EL_PM25_4, EL_PM25_5, EL_PM25_6


In [10]:
print("B Model:\n" + ", ".join(B_train.columns.tolist()))

B Model:
EL_PM25, LB_D_PM25, CH_D_PM25, NT_D_PM25


# Base Line

In [11]:
# Linear Model
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")
start_time = time.time()
L = LinearRegression()
L.fit(A_train, y_train)
L_list = L.predict(A_test)
L_rmse = sqrt(mean_squared_error(L_list, y_test))
end_time = time.time()
print("Time consume: %f s."%(end_time - start_time))
print("L rmse: ", round(L_rmse, 6))

Time consume: 0.100032 s.
L rmse:  4.719499


In [12]:
# Persistence Model
P_list=A_test["EL_PM25"]
P_rmse = sqrt(mean_squared_error(P_list, y_test))
print("P rmse: ", round(P_rmse, 6))

P rmse:  5.511364


# Ensembel Model

In [13]:
rmse_list_A =list()
rmse_list_B =list()
rmse_list_C =list()
t = time.time()
for _ in range(1):
    # A model
    A = neural_network.MLPRegressor()
    A.fit(A_train, y_train)
    A_preds = A.predict(A_test)
    A_rmse = sqrt(mean_squared_error(A_preds, y_test))
    rmse_list_A.append(A_rmse)
    
    # B model
    B = LinearRegression()
    B.fit(B_train, y_train)
    B_preds = B.predict(B_test)
    B_rmse = sqrt(mean_squared_error(B_preds, y_test))
    rmse_list_B.append(B_rmse)
    
    # C test data
    C_test = pd.DataFrame({'A_preds':A_preds, 'B_preds':B_preds})
    # C train data
    A_predict = A.predict(A_train)
    B_predict = B.predict(B_train)
    C_train = pd.DataFrame({'A_preds':A_predict, 'B_preds':B_predict})
    C_train_Feature = train[PM25 + meteorology_cols + wd_cols].reset_index(drop=True)
    C_test_Feature = test[PM25 + meteorology_cols + wd_cols].reset_index(drop=True)
    C_train = pd.concat([C_train, C_train_Feature], axis=1)
    C_test = pd.concat([C_test, C_test_Feature], axis=1)

    # C model
    C = xgboost.XGBRegressor()
    C.fit(C_train, y_train)
    C_list = C.predict(C_test)
    C_rmse = sqrt(mean_squared_error(C_list, y_test))
    rmse_list_C.append(C_rmse)
    
print("Time consume: %f s."%(time.time() - t))
print("A rmse: %f"%(sum(rmse_list_A)/float(len(rmse_list_A))))
print("B rmse: %f"%(sum(rmse_list_B)/float(len(rmse_list_B))))
print("C rmse: %f"%(sum(rmse_list_C)/float(len(rmse_list_C))))

Time consume: 5.965839 s.
A rmse: 4.543035
B rmse: 5.413168
C rmse: 4.558493


In [14]:
print("C Model:\n" + ", ".join(C_train.columns.tolist()))

C Model:
A_preds, B_preds, EL_PM25, EL_TEMP, EL_RH, EL_RAINFALL, EL_WS, LB_WD_W, CH_WD_W, NT_WD_W
